In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyfbs_cython as pyfbs

### Define parameters

In [ ]:
eosDD2 = pyfbs.PyEoStable("../EOS_tables/eos_HS_DD2_with_electrons.beta")
#eosCausal = pyfbs.PyCausalEoS(1e-10)

### Test single star integration

In [ ]:
mu = 1.0
lam = 0.
rho_c =  0.002
phi_c = 0.002

In [ ]:
myFBS = pyfbs.PyFermionBosonStar.FromParameters(eosDD2, mu, lambda_=lam, rho_0=rho_c, phi_0=phi_c)
myFBS.bisection(1., 10)
myFBS.get()
res = myFBS.evaluate_model()
myFBS.get()

In [ ]:
fig = plt.figure(figsize=(12,8)); ax = fig.gca()
myFBS.plot(ax)
r = res[:,0]
M_T = r /2. * (1. - 1./res[:,1]**2)
l, = ax.loglog(r, M_T, label='M_T')
ax.axhline(myFBS.get()['M_T'], linestyle='--', color=l.get_c())
ax.axvline(myFBS.get()['R_F'], linestyle='--')
ax.axvline(myFBS.get()['R_B'], linestyle='--')
ax.axvline(myFBS.get()['R_G'], linestyle='--')
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.set_ylim(bottom=1e-16, top=1e5)
ax.legend(); ax.grid()
print(r[-1])

In [ ]:
myFBS = pyfbs.PyFermionBosonStarTLN.FromFBS(myFBS)
myFBS.bisection_phi_1(1e-3 * phi_c, 1e5 * phi_c)
res = myFBS.evaluate_model()
myFBS.get()

In [ ]:
fig = plt.figure(figsize=(12,8)); ax = fig.gca()
myFBS.plot(ax)
r = res[:,0]
y = r * res[:,7]/res[:,6]
l, = ax.loglog(r, y, label='y')
ax.axhline(myFBS.get()['y_max'], linestyle='--', color=l.get_c())
ax.axvline(myFBS.get()['R_ext'], linestyle='--', color=l.get_c())
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.set_ylim(bottom=1e-8, top = 1e4)
ax.legend(); ax.grid()

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(8,6), sharex='all')
r = res[:,0]
y = r * res[:,7]/res[:,6]
phi = res[:,3]
P = res[:,5]
M_T = r/2. * (1. - 1./res[:,1]**2)
C = M_T/r
lambda_tidal = 16./15. * ((1.-2.*C)**2* (2. + 2.*C*(y-1.) - y)
                                                / (2.*C*(6. - 3.*y + 3.*C*(5.*y-8.))
                                                    + 4.*C**3*(13. - 11.*y + C*(3.*y-2.) + 2.*C*C*(1. + y))
                                                    + 3.* (1. - 2.*C)**2 *(2. - y + 2.*C*(y-1.))*np.log(1.-2.*C)))
#lambda_tidal1 = lambda_tidal*M_T**5
lambda_tidal1 = lambda_tidal*myFBS.get()['M_T']**5

axes[0].plot(r, phi, label="$\phi$")
axes[0].plot(r, P, label="$P$")
axes[0].set_yscale('log')
axes[0].set_xscale('log')
axes[0].set_xlim(left=1e-1, right=1e4)
axes[0].set_ylim(1e-8, 1)
axes[0].legend(fontsize=12); axes[0].grid()
axes[0].set_ylabel("components", fontsize=14)

l, = axes[1].plot(r, y, label='y')
axes[1].set_ylim(0,3)
axes[1].grid(); axes[1].set_ylabel("$y$", fontsize=16)
#plt.axhline(myFBS.get()['y_max'], linestyle='--', color=l.get_c())
axes[1].axvline(myFBS.get()['R_ext'], linestyle='--', color=l.get_c())


axes[2].plot(r, lambda_tidal1)
#axes[2].plot(r, lambda_tidal1, linestyle='--')
axes[2].axvline(myFBS.get()['R_ext'], linestyle='--', color=l.get_c())
axes[2].set_yscale('log')
#axes[2].set_ylim(1e2, 1e3)
axes[2].grid(); 
axes[2].set_ylabel("$\lambda_{tidal}$", fontsize=16)
axes[2].set_xlabel("r", fontsize=18)
axes[2].set_ylim(1e1, 1e3)
axes[2].set_xlim(1e0, 1e2)

fig.subplots_adjust(hspace=0.1)
#plt.savefig("y_extraction.pdf")

In [ ]:
dr = r[1:]-r[:-1] 
print(np.shape(dr), dr, np.max(dr), np.min(dr))

### Compare to Fig 2. of https://arxiv.org/pdf/1606.03035.pdf

In [ ]:
mu = 1.
lam = 0.
rho_c = np.array([0.])
phi_c = np.geomspace(3e-4, 1e-1, 50)
pMR = pyfbs.PyMRcurve.from_rhophi_list(mu, lam, eosDD2, rho_c, phi_c, "")
tln_curve = pyfbs.PyMRcurve.calc_TLN_curve(pMR)

In [ ]:
M = np.array(  [fbs.get()['M_T'] for fbs in tln_curve])
Lam = np.array([fbs.get()['lambda_tidal'] for fbs in tln_curve])

In [ ]:
plt.plot(phi_c, Lam*M**5)
plt.grid()
plt.ylabel("$\lambda_{tidal}$"); plt.ylim(bottom=0, top=1200)
plt.xlabel("$\phi_c$"); plt.xlim(left=0., right=0.05)

In [ ]:
# look at a specific instance
fig = plt.figure(figsize=(12,8)); ax = fig.gca()
fbs = tln_curve[0]
res = fbs.evaluate_model()

fbs.plot(plt.gca())
r = res[:,0]

M_T = r /2. * (1. - 1./res[:,1]**2)
C = M_T/r
lambda_tidal = 16./15. * ((1.-2.*C)**2* (2. + 2.*C*(y-1.) - y)
                                               / (2.*C*(6. - 3.*y + 3.*C*(5.*y-8.))
                                                    + 4.*C**3*(13. - 11.*y + C*(3.*y-2.) + 2.*C*C*(1. + y))
                                                    + 3.* (1. - 2.*C)**2 *(2. - y + 2.*C*(y-1.))*np.log(1.-2.*C)))

lambda_tidal = lambda_tidal*M_T**5

l, = ax.loglog(r, M_T, label='M_T')
ax.axhline(fbs.get()['M_T'], linestyle='--', color=l.get_c())

l, = ax.loglog(r, lambda_tidal, label='lambda_tidal')
ax.axhline(fbs.get()['lambda_tidal'], linestyle='--', color=l.get_c())

y = r * res[:,7]/res[:,6]
l, = ax.loglog(r,y, label='y')
ax.axvline(fbs.get()['R_ext'], linestyle='--', color=l.get_c())
ax.axhline(fbs.get()['y_max'], linestyle='--', color=l.get_c())

ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.set_ylim(bottom=1e-12)
ax.legend(); ax.grid()
fbs.get()

### Compare to DD2 k2

In [ ]:
rho_c = np.geomspace(1e-4, 1e-2, 88)
phi_c = np.array([0.])
pMR = pyfbs.PyMRcurve.from_rhophi_list(mu, lam, eosDD2, rho_c, phi_c, "")
tln_curve = pyfbs.PyMRcurve.calc_TLN_curve(pMR)

In [ ]:
M = np.array([fbs.get()['M_T'] for fbs in tln_curve])
R = np.array([fbs.get()['R_F'] for fbs in tln_curve])
k2 = np.array([fbs.get()['k2'] for fbs in tln_curve])
C = M/R / 1.477

In [ ]:
plt.plot(C, k2)
plt.grid()
plt.xlabel("C"); #plt.ylim(bottom=0, top=1200)
plt.ylabel("$k_2$"); #plt.xlim(left=0., right=0.05)